This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data. As a first step, you will need to register a free account on the http://www.quandl.com website. After you register, you will be provided with a unique API key. Simply copy the key from the site (below)

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'nkLWzTFsuAkSxhW1YJcA'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

**The Company**

- We will focus on _equities data from the **Frankfurt Stock Exhange (FSE)**
- Analyze stock prices of a company called **Carl Zeiss Meditec**
- Carl Zeiss Meditec manufactures tools for eye examinations as well as medical lasers for laser eye surgery: 
- Ticker **AFX_X**
- company website: https://www.zeiss.com/meditec/int/home.html. 


>**Quandl API instructions**
>>https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here:

>**Requests**
>>http://docs.python-requests.org/en/master/

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy:

>**Collections**
>>https://pymotw.com/3/collections/ ).

Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: 

>**Python data structures**
>>https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. 

>_**Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.**_

In [2]:
# First, import the relevant modules
import numpy as np
import requests
import collections

# check current status 
requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=API_KEY')

print(requests.status_codes)

print(requests.codes['temporary_redirect'])
 
print(requests.codes.teapot)

print(requests.codes['o/'])

<module 'requests.status_codes' from 'C:\\Users\\Emtma_000\\Anaconda3\\lib\\site-packages\\requests\\status_codes.py'>
307
418
None


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# ... into the JSON structure that will be returned
data=requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=API_KEY')

# Grab the whole year 2017 for AFX_X

In [4]:
# grab the whole year 2017 w/o collapse and transform args
myrequest = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?\
                         column_index=4&start_date=2017-01-01&end_date=2017-12-29&api_key=API_KEY')

In [5]:
# Inspect the JSON structure, note the nesting/structure
print(type(myrequest.json()))

<class 'dict'>


In [6]:
# check headers
myrequest.headers

{'Allow': 'GET, HEAD, POST, PUT, DELETE, OPTIONS, PATCH', 'Content-Type': 'application/json; charset=utf-8', 'Date': 'Fri, 12 Jun 2020 11:39:01 GMT', 'Server': 'openresty', 'Connection': 'keep-alive', 'Set-Cookie': 'visid_incap_2261005=WOjs8jwMQaiwbO3f3VAe4FVp414AAAAAQUIPAAAAAAANaJ57+skGvA/3aHSllaNE; expires=Fri, 11 Jun 2021 14:37:39 GMT; HttpOnly; path=/; Domain=.quandl.com; Secure; SameSite=None, nlbi_2261005=9j+WDhyrJQkXBxmJgSMXTwAAAACAw1czgBvLxpur9Gd1RamL; path=/; Domain=.quandl.com; Secure; SameSite=None, incap_ses_113_2261005=vFKQQDlVUlULlLW59HSRAVVp414AAAAAD3tDdpwJfxrVtNFYCUfQEQ==; path=/; Domain=.quandl.com; Secure; SameSite=None', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-CDN': 'Incapsula', 'Content-Encoding': 'gzip', 'Transfer-Encoding': 'chunked', 'X-Iinfo': '12-38197609-38197633 NNYY CT(0 0 0) RT(1591961941295 159) q(0 0 0 -1) r(0 0) U11'}

In [7]:
myrequest.headers['Content-Type']    # OR: myrequest.headers.get('content-type')

'application/json; charset=utf-8'

In [8]:
#check encoding type used default by requests
print(myrequest.encoding)

# Change type of encoding example
#myrequest.encoding = ‘ISO-8859-1’

utf-8


In [9]:
# calling text converts request obj to string form 
print('.text on req obj data-type is {}\n'.format(type(myrequest.text)))
#myrequest.text

# requests comes with a built-in JSON decoder
# ... note that json format is of dictionary type
print('calling .json() on request obj converts to', type(myrequest.json()))

.text on req obj data-type is <class 'str'>

calling .json() on request obj converts to <class 'dict'>


# convert request to dictionary

In [10]:
AFXX = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?\
                    start_date=2017-01-01&end_date=2017-12-31&api_key=nkLWzTFsuAkSxhW1YJcA")
# convert to dictionary and select "dataset" key
AFXX = AFXX.json()['dataset']  #dictionary = myrequest.json()['key']
print('column names for new query:\n{}'.format(AFXX['column_names']))

column names for new query:
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [11]:
labels = []
for x in AFXX:
    labels.append(x)
# all keys as list
display(labels[:7])

['id',
 'dataset_code',
 'database_code',
 'name',
 'description',
 'refreshed_at',
 'newest_available_date']

In [12]:
# select data by colling key/label/header name
# ... this also converts selection to a list
col_names = AFXX['column_names']
#Check-type: print('col_names object = {}\n'.format(type(col_names)))

data = AFXX.get('data'); print('data[0]:')
print(data[0],'\n\n')

# vals initial day for 2017 
val = [v for v in data[-1]]
print('(AFX_X) 2017 first day results:\n {}: {}\
\n {}: {}\n {}: {}\n {}: {}\n {}: {}\n {}: {}\n'.format(col_names[0], val[0], 
                                                 col_names[1], val[1] ,col_names[2], val[2], 
                                                 col_names[3], val[3], col_names[4], val[4],
                                                          col_names[5], val[5], col_names[6], val[6]))
#results for final trading day 2017
vals = [o for o in data[1]]
print('\nAFX_X Final trading day 2017 results:\n {}: {}\
\n {}: {}\n {}: {}\n {}: {}\n {}: {}\n {}: {}'.format(col_names[0], vals[0], 
                                                 col_names[1], vals[1] ,col_names[2], vals[2], 
                                                 col_names[3], vals[3], col_names[4], vals[4],
                                                 col_names[5], vals[5], col_names[6], vals[6]))

data[0]:
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None] 


(AFX_X) 2017 first day results:
 Date: 2000-06-07
 Open: None
 High: None
 Low: None
 Close: 38.0
 Change: None


AFX_X Final trading day 2017 results:
 Date: 2017-12-28
 Open: 51.65
 High: 51.82
 Low: 51.43
 Close: 51.6
 Change: None


# Calculate high and lowest opening price during the year

In [13]:
# build explicit lists of each for entire yr
dates = [d[0] for d in list(data)]
open_prices = [o[1] for o in data]
high_prices = [h[2] for h in data]
low_prices = [l[3] for l in data]
close_prices = [c[4] for c in data]

In [14]:
#opening price high
open_prices = np.array(open_prices).astype(np.float)
print('highest opening price, 2017: {}'.format(max(open_prices)))
#opening price low
print('Lowest opening price, 2017: {}'.format(min(open_prices)))

highest opening price, 2017: 53.11
Lowest opening price, 2017: 4.2


# Largest intraday change for 2017

In [15]:
price_flux = np.array(high_prices).astype(np.float) - np.array(low_prices).astype(np.float)
price_flux = price_flux.astype(np.float)

In [16]:
import pandas as pd

dates_ser = pd.Series(dates)
change_ser = pd.Series(price_flux)
dates_change = pd.concat([dates_ser, change_ser], 
                         axis=1).rename(columns={0:'date',
                                                 1:'price_flux'})
max_intraday_incr = dates_change.price_flux.max()
max_intraday_decr = dates_change.price_flux.min()
# greatest price change 2017
max_intraday_val = round(np.float(dates_change\
                                  [dates_change.price_flux == max_intraday_incr]\
                                  ['price_flux'].values), 2)
max_change_date = np.array(dates_change\
                           [dates_change.price_flux == max_intraday_incr]\
                           ['date']).item()
print('\nlargest intraday price increase was {} on {}'.format(max_intraday_val,
                                                              max_change_date))


largest intraday price increase was 4.39 on 2016-06-24


# What was the largest closing price change over 48 hours?

In [17]:
#dates_ser = pd.Series(dates)
close_ser = pd.Series(close_prices)
dates_close = pd.concat([dates_ser, 
                         close_ser], axis=1).rename(columns={0:'date', 
                                                             1:'close'})
# percent change process
# #round(dates_close.close.pct_change(),3)

changes = dates_close.close.diff(periods=-2)
largest_change = round(np.max(np.abs(changes)),2)

dates_close_change = pd.concat([dates_close.date, dates_close.close, changes], axis=1)
dates_close_change.columns = ['date', 'close', 'change']
# answer
print('the largest price change over a 2-day period for 2017 was a loss of ${}'.format(largest_change))
# span
dates_close_change[98:101]

the largest price change over a 2-day period for 2017 was a loss of $7.0


,date,close,change
98,2017-08-09,41.81,-3.15
99,2017-08-08,44.37,-0.70
100,2017-08-07,44.96,-0.01


# What was the average daily trading volume during 2017 for AFX_X?

In [18]:
data = AFXX.get('data')
traded_volume = [c[6] for c in data]
print('the average daily trade volume was', np.mean(traded_volume))

the average daily trade volume was 47040.76172649196
